In [110]:
import re
import pandas as pd
import MySQLdb
import sys, csv, os
sys.path.append("D:/DataBaseUpdate/To_clone")
from ConfigFiles import config #this attaches my database credentials

In [111]:
academic = []
government = []
corporate = []
hospital = []

lookup = {'Academic': academic, 'Government': government, "Corporate": corporate, 'Hospital': hospital}

def add_to_list(item, category):
    lookup[category].append(item)

with open('AcadCorpGovIndiv.txt', 'rb') as myfile:
    category = None
    for line in myfile.readlines():
        if line.startswith("*"):
            category = line[2:].strip("\r\n")
            if category == 'People':
                break
        if not line.startswith("*"):
            add_to_list(line[6:].strip("\r\n").replace("\\b", ""), category)

In [112]:
government += ["represented", "Department of","United States of America", "The National Institute of Standards and Technology",
               "National Institutes of Health", "Tennessee Valley Authority"]
academic += ['Massachusetts Institute of Technology', 'California Institute of Technology','Rensselaer Polytechnic Institute']
hospital += ['Sloan-Kettering Institute for Cancer Research']
corporate += ['Battelle Memorial Institute', 'The Scripps Research Institute','The Salk Institute for Biological Studies','Dana-Farber Cancer Institute, Inc.', 'Midwest Research Institute', 'Southwest Research Institute', 'Whitehead Institute for Biomedical Research', 'SRI International', 'International Business Machines', 'General Dynamics']

In [113]:
#adding in the logic from Sanjay's email
with open('new_thesaurus.txt', 'rb') as myfile:
    counter =0
    for line in myfile.readlines():
        if line != "\r\n":
            if counter ==0:
                academic += [item.lstrip(" ").rstrip(" ").strip("\r\n") for item in line.split(',')]
            if counter ==1:
                government += [item.lstrip(" ").rstrip(" ").strip("\r\n") for item in line.split(',')]
            if counter ==2:
                corporate += [item.lstrip(" ").rstrip(" ").strip("\r\n") for item in line.split(',')]
            counter +=1

print "corporate"
print corporate

print "academic"
print academic

print "hospital"
print hospital

print "government"
print government

corporate
['AB$', 'AG', 'Boeing', 'Company', 'Consultant', 'Corp', 'Gmbh', 'IBM', 'Inc', 'KGAA', 'KK$', 'LLC', 'Limited', 'Ltd', 'NTT', 'NV$', 'PLC', 'SA$', 'SPA$', 'Battelle Memorial Institute', 'The Scripps Research Institute', 'The Salk Institute for Biological Studies', 'Dana-Farber Cancer Institute, Inc.', 'Midwest Research Institute', 'Southwest Research Institute', 'Whitehead Institute for Biomedical Research', 'SRI International', 'International Business Machines', 'General Dynamics', 'Inc', 'Group', 'Foundation', 'Co', 'limited', 'LTD', 'LLC', 'Corp', 'Company', 'LP', 'LLP ']
academic
['Acad', 'Academic', 'Coll', 'Colleg', 'Ecole', 'Escola', 'Inst of Tech[a-z]+', 'MIT', 'Sch', 'School', 'Univ', 'Massachusetts Institute of Technology', 'California Institute of Technology', 'Rensselaer Polytechnic Institute', 'university', 'alumni', 'univ', 'national cancer', 'brigham', 'jackson lab', 'research center', 'akademie', 'vib', 'RIKEN', 'Eye & Ear', 'medical school', 'national jewish 

In [114]:
academic_stops = [item if "$" in item else (item + "\\b") for item in academic ]
academic_stops = ["\\b" + item for item in academic_stops ]

corporate_stops = [item if "$" in item else (item + "\\b") for item in corporate ]
corporate_stops = ["\\b" + item for item in corporate_stops ]

government_stops = [item if "$" in item else (item + "\\b") for item in government ]
government_stops = ["\\b" + item for item in government_stops ]

hospital_stops = [item if "$" in item else (item + "\\b") for item in academic ]
hospital_stops = ["\\b" + item for item in hospital_stops ]

print "corporate"
print corporate_stops

print "academic"
print academic_stops

print "hospital"
print hospital_stops

print "government"
print government_stops

corporate
['\\bAB$', '\\bAG\\b', '\\bBoeing\\b', '\\bCompany\\b', '\\bConsultant\\b', '\\bCorp\\b', '\\bGmbh\\b', '\\bIBM\\b', '\\bInc\\b', '\\bKGAA\\b', '\\bKK$', '\\bLLC\\b', '\\bLimited\\b', '\\bLtd\\b', '\\bNTT\\b', '\\bNV$', '\\bPLC\\b', '\\bSA$', '\\bSPA$', '\\bBattelle Memorial Institute\\b', '\\bThe Scripps Research Institute\\b', '\\bThe Salk Institute for Biological Studies\\b', '\\bDana-Farber Cancer Institute, Inc.\\b', '\\bMidwest Research Institute\\b', '\\bSouthwest Research Institute\\b', '\\bWhitehead Institute for Biomedical Research\\b', '\\bSRI International\\b', '\\bInternational Business Machines\\b', '\\bGeneral Dynamics\\b', '\\bInc\\b', '\\bGroup\\b', '\\bFoundation\\b', '\\bCo\\b', '\\blimited\\b', '\\bLTD\\b', '\\bLLC\\b', '\\bCorp\\b', '\\bCompany\\b', '\\bLP\\b', '\\bLLP \\b']
academic
['\\bAcad\\b', '\\bAcademic\\b', '\\bColl\\b', '\\bColleg\\b', '\\bEcole\\b', '\\bEscola\\b', '\\bInst of Tech[a-z]+\\b', '\\bMIT\\b', '\\bSch\\b', '\\bSchool\\b', '\\bUniv\\

In [120]:
academic_regex = re.compile("|".join(academic), re.IGNORECASE)

In [121]:
government_regex = re.compile("|".join(government), re.IGNORECASE)

In [122]:
corporate_regex = re.compile("|".join(corporate), re.IGNORECASE)
hospital_regex = re.compile("|".join(hospital), re.IGNORECASE)

institute_regex = re.compile("institute", re.IGNORECASE)

In [127]:
def org_types(text):
    if text == "":
        return "Person"
    academic = bool(re.search(academic_regex, text))
    government = bool(re.search(government_regex, text))
    corporate = bool(re.search(corporate_regex, text))
    hospital = bool(re.search(hospital_regex, text))
    corp_institute = bool(re.search(institute_regex, text)) & corporate & academic 

    if corp_institute:
        return "Corporate"
    if academic: 
        return "Academic"
    if government:
        return "Government"
    if hospital:
        return "Hospital"
    if corporate:
        return "Corporate"
    else:
        return "Ambiguous"

In [128]:
data = pd.read_csv('F:\Govt_Int\Final_CSVS/assignee_type.csv')
data['organization'] = data['organization'].fillna("")
data = data.dropna()
data['thes_types'] = data['organization'].apply(lambda x : org_types(x))
print data

       patent_id  assignee_type  \
0        3930295            2.0   
1        3930295            2.0   
2        3930366            2.0   
3        3930405            6.0   
4        3930449            6.0   
5        3930537            6.0   
6        3930541            6.0   
7        3930550            6.0   
8        3930628            6.0   
10       3930735            6.0   
12       3930848            2.0   
13       3930956            2.0   
15       3931325            2.0   
16       3931354            2.0   
17       3931388            6.0   
18       3931447            6.0   
19       3931456            6.0   
20       3931462            6.0   
21       3931513            2.0   
23       3931522            6.0   
24       3931532            6.0   
25       3931607            6.0   
26       3931687            2.0   
27       3932014            6.0   
28       3932123            6.0   
29       3932226            2.0   
30       3932262            2.0   
31       3932300    

In [41]:
#data["Inst"] = data['organization'].apply(lambda x : True if 'Institute' in x else False)
#data[data["Inst"]==True].groupby('organization').max().sort_values("thes_types", ascending=False)

In [129]:
data.to_csv('F:\Govt_Int\Final_CSVS/assignees_lookedup_types.csv')

### Check the most common assignees

In [130]:
data = pd.read_csv('F:\Govt_Int\Final_CSVS/assignees_lookedup_types.csv')
data.head()

,Unnamed: 0,patent_id,assignee_type,organization,thes_types
0,0,3930295,2.0,Signetics Corporation,Corporate
1,1,3930295,2.0,Watkins-Johnson,Ambiguous
2,2,3930366,2.0,General Motors Corporation,Corporate
3,3,3930405,6.0,The United States of America as represented by...,Government
4,4,3930449,6.0,The United States of America as represented by...,Government


In [89]:
counts = data.groupby("organization").agg({'patent_id':'count', 'thes_types':'min'})
to_check = counts[counts['patent_id'] >20]
#to_check['thes_types'].value_counts()

In [90]:
to_check.reset_index(inplace=True)
to_check.columns = ['organization', 'type', 'count']
to_check.head(20)
#to_check.to_csv('F:/Govt_Int/ManualCheck/top_orgs_to_check.csv', index = False)

,organization,type,count
0,3M Innovative Properties Company,Corporate,143
1,"AIR PRODUCTS AND CHEMICALS, INC.",Corporate,110
2,ARCH Development Corporation,Corporate,255
3,AT&T Bell Laboratories,Corporate,23
4,Abbott Laboratories,Corporate,41
5,"Abuzz Technologies, Inc.",Corporate,27
6,"Advanced Ceramics Research, Inc.",Corporate,21
7,"Advanced Fuel Research, Inc.",Corporate,25
8,"Advanced Liquid Logic, Inc.",Corporate,75
9,"Advanced Micro Devices, Inc.",Corporate,37


### Use the regex to classify all the assignees

In [14]:
all_assignees = pd.read_csv("F:/Govt_Int/Final_CSVS/all_assignees.csv").fillna("")
all_assignees['thes_types'] = all_assignees['organization'].apply(lambda x : org_types(x))
all_assignees.head()

,id,type,name_first,name_last,organization,thes_types
0,00003f5c5d92354570b9bae01b3e7272,0,,,"E-Z Anchor Bolt Template, Inc.",Corporate
1,000058159cdbf11415ffad8001c192f6,0,,,Roll and Go AG,Corporate
2,00005a7b722c0d70af7c0f6706cdc067,0,,,PERSCITUS INNOVATIONS LLC,Corporate
3,00007b72e8f46f63797266c63ad3a34f,0,,,"Fleetwood Aluminum Products, Inc.",Corporate
4,000099cec50350524fc0704f0cdd8db6,0,,,B.V. Nederlandse Industrie Van Eiprodukten,Corporate


In [15]:
top_orgs = pd.read_csv("F:/Govt_Int/Final_CSVS/top_orgs_to_check_checked.csv")
top_orgs.head()

,organization,type,count,updated_type,check_flag,NonProfit
0,3M Innovative Properties Company,Corporate,139,NaN,NaN,NaN
1,"AIR PRODUCTS AND CHEMICALS, INC.",Corporate,102,NaN,NaN,NaN
2,ARCH Development Corporation,Corporate,261,NaN,NaN,NaN
3,AT&T Bell Laboratories,Corporate,23,NaN,NaN,NaN
4,"ATC Technologies, Inc.",Corporate,26,NaN,NaN,NaN


In [16]:
dfmerged = pd.merge(top_orgs, all_assignees, on = "organization", how = "outer")

In [17]:
dfmerged['final_type'] = dfmerged.apply(lambda row : row['updated_type'] if not pd.isnull(row['updated_type']) else row['thes_types'],axis=1) 

In [18]:
dfmerged = dfmerged[['id', 'type_y', 'name_first', 'name_last', 'organization', 'final_type']]
dfmerged.rename({'type_y': 'type', 'final_type': 'thes_types'}, inplace = True)
dfmerged.head()

,id,type_y,name_first,name_last,organization,final_type
0,358d3cb746241ec1105de0542e47fbcd,0.0,,,3M Innovative Properties Company,Corporate
1,aca8358f38f9eb71191a353a59a96ea9,0.0,,,"AIR PRODUCTS AND CHEMICALS, INC.",Corporate
2,610f73966a02992c651f46e9d0490d62,0.0,,,ARCH Development Corporation,Corporate
3,3a95e99e9d014e0f405159dfb75ff537,0.0,,,AT&T Bell Laboratories,Corporate
4,25c424953c244978383e1fe39d8c1a8c,0.0,,,"ATC Technologies, Inc.",Corporate


In [19]:
all_assignees['thes_types'].value_counts()

Corporate     322672
Person         40483
Ambiguous       6508
Academic        6320
Government       688
Hospital         242
Name: thes_types, dtype: int64

In [20]:
mydb = MySQLdb.connect(config.host, config.username, config.password, 'patent_20170808')

In [21]:
#only run this once to upload the data
all_assignees.to_sql(con=mydb, name='assignees_by_type_v2', if_exists='replace', flavor='mysql')

C:\Program Files\Anaconda2\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 67
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 74
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 171
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 179
  conn.executemany(self.insert_statemen